Procesamiento de imagenes

Idea es procesar imagenes, para nosotros una imagen va a ser en w * h  * 3

w = width
h = height
3 = los tres canales del color (RGB)

Tipo de problemas que podemos tener:

- Clasificacion: tenemos una imagen y queremos decir a que clase pertenece. Para clasificacion los datasets mas populares son Mnist (son datasets de deteccion de digitos manuscritos, generalmente del 1 al 9, 50.000 imagenes, 10 clases (10 digitos posibles)), Cifar (60.000 imagenes de 32 x 32 en 10 clases de deteccion de objetos de imagenes a color), ImageNet (20.000 clases). Cuando uno prueba algoritmos de clasificacion de imagenes, Mnist es bastante limitado, Cifar es un poco mas completo, el de ImageNet es el mejor (son los que le importa a la comunidad cientifica, es como el banco de pruebas). En general, los mejores modelos de CV para imagenet funcionan mejor que los humanos. Los humanos tienen un 94% de accuracy mientras que ImageNet tiene 96%.

- Object detection: tenemos una imagen y queremos encontrar objetos en una imagen.

Clasificacion:

Objetivo: Ver una imagen y clasificarla a una o n clases. 

Primer approach: extraccion de features o caracteristicas de una imagen. Dada una imagen la quiero convertir en un vector de features y una vez que tengo eso utilizo algun modelo de ML que ya conocemos, por ej random forest, xgboost, redes neuronales, cualquier algoritmo que funcione para multiclases. Como armo el vector? podemos agarrar cada bit y decir que uno vale el promedio del canal rojo, otro el promedio del canal verde, otro el promedio del canal azul, tambien se podria usar el maximo del rojo, el maximo del verde, el maximo del azul, la varianza de cada color.

Features de imagenes:

- Estadisticos: promedios, maximos, desviacion estandar.
- Puntos: cuales son los puntos mas importantes de una imagen, no tomar el valor del pixel sino un descriptor que me descirba esos puntos. para eso se puede usar SIFT que genera descritpores de 128 bytes para cada punto clave de la imagen. La deteccion de putnos clave la hace con el algoritmo de SIFT. Este algoritmo hasta el 2020 estaba bajo un tema de patente, a partir de ahi expiro y se conviritio en libre de usar. SIFT se usa por ejemplo en construccion de imagenes panoramicas. Lo que hace SIFT es obtiene los mismos descriptores de dos imagenes distitnas que comparten algo y luego los une en el punto que se unen. Por ejemplo una montania divida en dos partes agarra los descriptores de un lado y del otro y la parte que es igual la une.
- Filtros: Nos interesan un poco mas, miramos la operacion de convolusion. La idea es tener un filtro de tamanio mejor que la imagen.
10  5   2   10  20              1       1       1
5   0   0   3   2               1       1       1
20  25  24  25  20              1       1       1
1   2   0   0   1
10  15  20  25  20

la convolucion de estas matrices me queda:

71  94  108
77  79  75
117 126 135




10  5   2   10  20              -1      -2      -1
5   0   0   3   2                0       0       0
20  25  24  25  20               1       2       1
1   2   0   0   1
10  15  20  25  20                              

La convolucion de estas matrices me queda:

72   79    52
0    -1    -7
-34  -18   -4


20  20  20  20                  -1      -2      -1
20  20  20  20                   0       0       0
0   0   0   0                    1       2       1
0   0   0   0

La convolucion de estas matrices queda:

-80     -80
-80     -80


20  20  20  20                   -1     -2      -1
20  20  20  20                    0      0       0
20  20  20  20                    1      2       1
20  20  20  20

la convolucion me queda:

0       0
0       0

20  20  0   0                   -1      -2      -1
20  20  0   0                    0       0       0      
20  20  0   0                    1       2       1
20  20  0   0

la convolucion me queda:

0       0
0       0

cuando tenemos una division de numeros tenemos un borde


0   0   20  20                  -1      0       1
0   0   20  20                  -2      0       2
0   0   20  20                  -1      0       1
0   0   20  20

la convolucion me queda:

80     80
80     80

0   0   0   0                   
0   0   0   0
20  20  20  20
20  20  20  20

80  80

80  80



sobel filter sirve para encontrar los 4 bordes posibles, horizontal vertical, etc

-1  -2  -1                  1   2   1
0   0   0                   0   0   0
1   2   1                   -1  -2  -1

-1  0   1                   1   0   -1  
-2  0   2                   2   0   -2
-1  0   1                   1   0   -1


hay cientos de estos filtros.

Porque no aplicar NNs? el prblema esta en la capa input, habria que hacer cada pixel con cada neurona lo que hace imposible la computabilidad, explota la cantidad de parametros lo que terminaria en overfitting.

Cuando emepzo a funcionar? cuando aparecieron las redes convolucionales. Son las mas importantes paa clasificacion de imagenes.

La idea de las redes convolucionales es simple pero fue una revolucion, hasta que aparecerion lo unico que funcionaba era extraer los features y armar un estadistico gigante. Esto se llama el problema de representacion. Las redes resuelven el problema de repsresntacion y nos permiten represtnar la imagen en crudo. Tomamos la imagen en crudo y los filtros los aprende la red, ese es el gran truco. En lugar de decirle aca voy a tener este filtro o este otro, la red directamente aprende cual es el filtro que se necesitan para hacer eso. Eso es una CNN. Las redes convolucionales empiezan con uno o mas filtros de convolucion en una imagen que esta en crudo. La funcion final aplicada es una softmax porque es un problema de multiclase.

Capas convolucionales (conv 2D)

input: n x n x c (imagen de n x n) (c = cantidad de canales)

output: piso(((w - k + 2*p) / S)) + 1
w = width
K = filter size
P = padding
S = Stride

Hiperparametos: 

- Tamanio del filtro: k (k * k * c)
- Cantidad de filtros: f (cuantos filtros hay que encontrar)
- Padding: tiene que ver con si quiero paddear la imagen, padding es agregarle un borde a la imagen con ceros. Dos opciones: o no usar padding, o usar same.
- Stride: Cuantos pixeles voy desplazando a la convolucion, es decir cuando multiplico por el filtro la matriz me muevo dependiendo del valor de S, esto va a cambiar la dimension de salida de la matriz, nos podemos mover de a 1, 2, 3.. etc pixeles por la imagen.
- Dilation: 

Si tengo una imagen de 5 x 5 y le aplico una convolucion de 2 x 2 el resultado me queda de 4 x 4, si le aplico una de 3 x 3 me queda de 3 x 3, si le aplico una de 4 x 4 me queda de 2 x 2, si es de 5 x 5 me queda de 1 x 1. la formula entonces si el input es n x n x c y el filtro es de k x k x c el output es: piso(((w - k + 2*p) / S)) + 1


suponemos que tenemos una imagen de 200 x 200 x 3 le aplico una conv 2D con k = 5, f = 16.

El output de los filtros es 196 x 196 x 16. 196 viene de (200 - 5) + 1. y 16 es porque son 16 filtros.

parametros: ((5 x 5 x 3) + 1) x 16. 5 x 5 x 3 es el tamanio del filtro y el +1 es el bias o termino independiente.

Las convoluciones siempre tienen la misma profundidad que tenia la imagen original.

si agrego el padding en vez de quedarme 196 x 196 x 16 me quedaria 200 x 200 x 16


Una red neuronal para una clasificacion podria ser:

input                       conv 2D (2D porque son imagenes, si fuese en video seria conv 3D)       conv 2D

200 x 200 x 3               K = 5                                                                   k = 3
                            F = 16      --------> 196 x 196 x 16 ------------------->               F = 32 -------------> 194 x 194 x 32
                            S = 1                                                                   S = 1   
                            padding = none                                                          padding = none
                            params:                                                                 params:
                             ((5 x 5 x 3) + 1) * 16                                                 ((3 x 3 x 16) + 1) x 32


capas max pooling 

20  20  10  0
0   10  5   0           maxpool 2 x 2       --->    20  10
2   4   -1  -2                                      6    3  ------> queda la dimensionalidad a la mitad
6   4   3   1

4 x 4

por ejemplo si le aplico max pool a 194 x 194 x 32 me queda 97 x 97 x 32.

en la capa de max pool tiene que aprender 0 parametros ya que no agregan parametros.

ahora si a 97 x 97 le aplico una conv 2D con k = 10, F = 6, stride = 2, pading = none, sale de output: 44 x 44 x 6, y la cantidad de params = ((10 x 10 x 32) + 1) x 6




Primer CNN

LeNet-5 (para MNIST):

- Input: imagenes de 32x32x1 porque estan en blanco y negro, cada pixel es blanco o negro.

CAPAS                     OUTPUT            PARAMS

conv2D 5x5 (6)            28x28x6          (5x5 + 1) x 6

avgPOOL                   14x14x6           ninguno para aprender (solo toma el promedio)

conv2D 5x5 (16)           10x10x16          (5x5x6 + 1) x 16

avgPOOL                   5x5x16            ninguno para aprender (solo toma el promedio)

conv2D 1x1 (120)          5x5x120           (1x1x16 + 1) x120

densa 120                   120                120x400 + 120

densa 84                    84                  120x84 + 84

softmax 10                  10                  10x84 + 10

en LeNet-5 las conv2D usan como funcion de activacion la tangete hiperbolica, asi como se usa ReLU para otros algos.



VGG-16 (una de las redes mas tipicas de computer vision)

Basicamente se entrena con ImageNet, y la salida puede ser cualquier cosa.

input: 224x224x3, ese es el tamanio de las imagenes que tienen en VGG-16, si no es de ese tamanio se hace un rezising

params = 0

CAPA                                                          OUTPUT                  params

conv2D 3x3 (64), padding="same", funcion de act=relu          224x224x64              ((3x3x3) + 1) x 64 += 1792

conv2D 3x3 (64), padding="same", funcion de act=relu          224x224x64              ((3x3x64) + 1) x 64 += 36928

maxPOOL                                                       112x112x64                ninguno

conv2D 3x3 (128), same, relu                                  112x112x128              ((3x3x64) + 1) x 128 += 73856

conv2D 3x3 (128)   ""   ""                                    112X112X128               147584

maxPOOl            ""   ""                                    56x56x128                     ninguno

3x3 (256)           ""  ""                                    56x56x256                 295168

3x3 (256)           ""  ""                                    56x56x256                 590080

3x3 (256)           ""  ""                                    56x56x256                 590080

maxpool             ""  ""                                    28x28x256                 ninguno

3x3 (512)           ""  ""                                    28x28x512                 1180160

3x3 (512)           ""  ""                                    28x28x512                 2359808

3x3 (512)           ""  ""                                    28x28x512                 2359808

maxpool             ""  ""                                    14x14x512                 ninguno

3x3 (512)           ""  ""                                    14x14x512                 2359808

3x3 (512)           ""  ""                                    14x14x512                 2359808

3x3 (512)           ""  ""                                    14x14x512                 2359808

maxpool             ""  ""                                    7x7x512                   ninguno

flatten             ""  ""                                    1x25088

densa 4096          ""  ""                                    1x4096                    4096x35039 = 102764544

densa 4096          ""  ""                                    1x4096                    4096x4097 = 16781312

softmax

params = 224 millones

Object Detection

la idea es: dada una imagen que me diga lo que hay. Encontrar las cosas que hay en una imagen, se pueden encontrar distintos objetos que son de distintas claes y ademas decirte en donde esta el objeto.

YOLO ("You only look once") (va por la version 8 (2023))

vamos a ver la v1 (2015), es la que sento las bases y luego se fue mejorando

convenciones: 

yolo va a definir un bounding box con cuatro numeros que son el centro y el ancho, cx, cy, wx, wy y todos estos van a estar entre 0 y 1. esto hace qe los bounding boxes sean independientes del tamanio de la imagen.

yolo tambien va a definir la probabilidad de deteccion (p) que va a ser un numero entre 0 y 1 tambien.

yolo tambien va a definir la probabilidad de cada clase (pc1 (car), pc2 (person), pc3 (etc)..., pcm)

los 4 numeros del bounding box, la proba de deteccion y la proba de cada clase es el vector de salida del yolo , es el y del yolo. 

yolo lo que hace es la imagen dividirla en una grilla de sxs y cada una de esas grillas tiene uno de esos vectores asociados. para yolo v1 la grilla es de 7x7 y las clases son 10. solo podia detectar 10 clases entonces el vector de salida es de 7x7x(5 + 10)x2 = 7x7x30. el 2 es porque hay 2 boundig boxes por cada grilla. El gran truco de Yolo es que es un regresor no hay funcion de activacion en la capa final, son todos numeros que van entre 0 y 1, al ser un regresor va a entrenar y predecir super rapido. entonces es importante saber que yolo es un regresor. para entrenarlo hay que darle por cada imagen el vector de 7x7x30. cuando P es 0, yolo todo lo demas lo ignora, la funcion de loss de yolo es una funcion partida que depende de P, cuando P es 1 calcula el loss para todo los demas (pc1, pc2, etc..)

